# Extracting embedding features from face data
In this notebook, we aim to extract embedding features from images using face recogntion extractors.
As an example, we use MOBIO dataset, and extract Arcface features from the face images:

In [ ]:
##### CHANGE YOUR DATABASE HERE
from bob.bio.face.config.database.mobio_male import database
annotation_type = database.annotation_type
fixed_positions = database.fixed_positions
memory_demanding = True
dask_client = None

In [ ]:
from bob.bio.face.embeddings.mxnet import arcface_insightFace_lresnet100
pipeline = arcface_insightFace_lresnet100(annotation_type=annotation_type,
                                          fixed_positions=fixed_positions,
                                          memory_demanding=memory_demanding)
transformer = pipeline.transformer

In [ ]:
from bob.pipelines import wrap


features_dir = "features" #Path to store extracted features
# Wrapping with CHECKPOINT and DASK
transformer = wrap(["checkpoint","dask"],
                   transformer,
                   features_dir=features_dir)

# Printing the setup of the transformer
print(transformer)

As an example, we consider 10 samples from this database and extract features for these samples:

In [ ]:
# get 10 samples from database
samples = database.all_samples()[:10]

In [ ]:
# Setting the DASK client
# HERE MAKE ABSOLUTELLY SURE THAT YOU DO `SETSHELL grid` 
# BEFORE STARTING THE NOTEBOOK

from dask.distributed import Client
from bob.pipelines.distributed.sge import SGEMultipleQueuesCluster

cluster = SGEMultipleQueuesCluster(min_jobs=1)
dask_client = Client(cluster)

In [ ]:

features = transformer.transform(samples)
if dask_client is not None:
    features = features.compute(scheduler=dask_client)


In the following cells, we convert the extracted features to `numpy.array` and check the size of features.

In [ ]:
import numpy as np
from bob.pipelines import SampleBatch

np_features = np.array(SampleBatch(features))

In [ ]:
np_features

In [ ]:
# KILL THE SGE WORKERS
dask_client.shutdown()